In [1]:

import pandas as pd
import numpy as np
import copy
import pickle
import time
import matplotlib.pyplot as plt
from IPython.display import display
from datetime import datetime
import datetime as dt
import ib_insync
print(ib_insync.__all__)

from ib_insync import *
#Andy - not using Jupyter
util.startLoop()

#Andy- this is the default account, setting so that orders can be allocated to it.
# acc_num = 'DU1870227'
acc_num = ''

#Connect to IB Gateway
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=1,timeout=100)


['util', 'Event', 'SoftDollarTier', 'PriceIncrement', 'Execution', 'CommissionReport', 'BarList', 'BarDataList', 'RealTimeBarList', 'BarData', 'RealTimeBar', 'HistogramData', 'NewsProvider', 'DepthMktDataDescription', 'ScannerSubscription', 'ScanDataList', 'FundamentalRatios', 'ExecutionFilter', 'PnL', 'PnLSingle', 'AccountValue', 'TickData', 'TickByTickAllLast', 'TickByTickBidAsk', 'TickByTickMidPoint', 'HistoricalTick', 'HistoricalTickBidAsk', 'HistoricalTickLast', 'TickAttrib', 'TickAttribBidAsk', 'TickAttribLast', 'MktDepthData', 'DOMLevel', 'TradeLogEntry', 'FamilyCode', 'SmartComponent', 'PortfolioItem', 'Position', 'Fill', 'OptionComputation', 'OptionChain', 'Dividends', 'NewsArticle', 'HistoricalNews', 'NewsTick', 'NewsBulletin', 'ConnectionStats', 'Contract', 'Stock', 'Option', 'Future', 'ContFuture', 'Forex', 'Index', 'CFD', 'Commodity', 'Bond', 'FuturesOption', 'MutualFund', 'Warrant', 'Bag', 'TagValue', 'ComboLeg', 'DeltaNeutralContract', 'ContractDetails', 'ContractDescrip

<IB connected to 127.0.0.1:7497 clientId=1>

In [8]:
contract = Stock("AAPL","SMART","USD")

In [9]:
# contract = Forex('EURUSD')
bars = ib.reqHistoricalData(
    contract, endDateTime='', durationStr='30 D',
    barSizeSetting='1 hour', whatToShow='MIDPOINT', useRTH=True)

Error 162, reqId 2819: Historical Market Data Service error message:No market data permissions for ISLAND STK, contract: Stock(symbol='AAPL', exchange='SMART', currency='USD')


In [10]:
bars

[]

In [8]:
ib.accountSummary()

[AccountValue(account='DF2280940', tag='AccountType', value='PARTNERSHIP', currency='', modelCode=''),
 AccountValue(account='DF2280940', tag='Cushion', value='1', currency='', modelCode=''),
 AccountValue(account='DF2280940', tag='LookAheadNextChange', value='0', currency='', modelCode=''),
 AccountValue(account='DF2280940', tag='AccruedCash', value='0.00', currency='USD', modelCode=''),
 AccountValue(account='DF2280940', tag='AvailableFunds', value='0.00', currency='USD', modelCode=''),
 AccountValue(account='DF2280940', tag='BuyingPower', value='0.00', currency='USD', modelCode=''),
 AccountValue(account='DF2280940', tag='EquityWithLoanValue', value='0.00', currency='USD', modelCode=''),
 AccountValue(account='DF2280940', tag='ExcessLiquidity', value='0.00', currency='USD', modelCode=''),
 AccountValue(account='DF2280940', tag='FullAvailableFunds', value='0.00', currency='USD', modelCode=''),
 AccountValue(account='DF2280940', tag='FullExcessLiquidity', value='0.00', currency='USD',

Peer closed connection


In [6]:
ib.

In [11]:
def format_days(x):
    if len(x)<2:
        return("0"+x)
    else:
        return(x)
    
def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3 

In [3]:
## Build Current Portfolio
CurrentPortfolio = pd.DataFrame()
for element in ib.positions(account=acc_num):
    symbol = str(element).split("symbol='")[1].split("'")[0]
    position =str(element).split("position=")[1].split(",")[0]
    list_row = {"symbol":symbol, "position":position}
    
    CurrentPortfolio = CurrentPortfolio.append(list_row, ignore_index=True)
    
CurrentPortfolio.index = CurrentPortfolio["symbol"]
CurrentPortfolio = CurrentPortfolio.transpose()
CurrentPortfolio = CurrentPortfolio[0:1]
CurrentPortfolio = CurrentPortfolio.astype(float).astype(int)
PreviousPortfolio = CurrentPortfolio.copy()
PreviousPortfolio = PreviousPortfolio[0:1]



In [4]:
CurrentPortfolio

symbol,KEY,KLAC,TOT,AXP,TOL,KGC,LYFT,TRGP,WRB,BUD,...,ROL,EWBC,MPC,LBRDK,RJF,GGG,PE,PCTY,TRU,RF
position,-302,-14,87,-16,60,-570,-43,-109,31,17,...,66,12,-56,18,-5,28,-239,5,13,-312


In [7]:
#TargetPortfolio
target_position = pd.read_csv("/home/sevder/ibtest/Alphas/Alpha5/2020_6_12.csv",index_col='Date')
target_position.index = pd.to_datetime(target_position.index)
target_position = target_position[target_position.index==target_position.index[-1]]
target_position = target_position.round(decimals=0)
target_position = target_position.fillna(0).astype(np.int64)

In [7]:
#Outputs
CurrentPortfolio.index = target_position.index
PreviousPortfolio.index = target_position.index

CurrentPortfolio.to_csv("/home/sevder/ibtest/current_run/CurrentPortfolio.csv")
PreviousPortfolio.to_csv("/home/sevder/ibtest/current_run/PreviousPortfolio.csv")
target_position.to_csv("/home/sevder/ibtest/current_run/TargetPosition.csv")


In [8]:
# Extra positions
inexistent_positions = set(target_position.columns) - set(CurrentPortfolio.columns)
inexistent_positions = pd.DataFrame(inexistent_positions)
inexistent_positions.columns = {"symbol"}
inexistent_positions.to_csv("/home/sevder/ibtest/current_run/inexistent_positions.csv")


In [9]:
#in target, but not in Current.
for element in inexistent_positions["symbol"]:
    CurrentPortfolio[element]=0

CurrentPortfolio.to_csv("/home/sevder/ibtest/current_run/new_CurrentPortfolio.csv")
    
#to be closed:
#set(CurrentPortfolio.columns)-set(target_position.columns)

In [10]:
#ExchangeData + Strategy Day
ExchangeData = pd.read_csv("exchange.csv")
ExchangeData.index= ExchangeData["Stock"]
#Test line
#print(ExchangeData.loc["AAPL"]["primaryExchange"]=="NASDAQ")
StrategyDay = str(target_position.index[0].year)+"-"+format_days(str(target_position.index[0].month))+"-"+format_days(str(target_position.index[0].day))


In [259]:
#Define Excluded Companies
excluded_companies = ["MLNX"]


#Orders
orders = target_position - CurrentPortfolio
execution_log=pd.DataFrame()
excluded_log=pd.DataFrame()
for column in orders.columns:
    if column not in excluded_companies:
    #     print(column)
        trade_amount = orders[column][0]
        company = column
        if trade_amount > 0:
            direction = "BUY"
        elif trade_amount < 0:
            direction = "SELL"
        elif trade_amount == 0:
            direction = "HOLD"

        trade_amount = abs(trade_amount)

    #EXECUTE        
        if direction!="HOLD":
            print(column)
            print(direction)
            try:
                contract = Stock(company, 'SMART', 'USD', primaryExchange=ExchangeData.loc[company]["primaryExchange"])
                order = MarketOrder(direction, trade_amount,account=acc_num) #,account=acc_num
    #             trade = ib.placeOrder(contract, order)
                ib.sleep(0.2) # (0.2)
                execution_line = {"symbol":column,"direction":direction,"contract":contract,"order":order}
                execution_log = execution_log.append(execution_line, ignore_index=True)
            except:
                execution_line = {"symbol":column,"direction":direction,"contract":"ERROR","order":"ERROR"}
                execution_log = execution_log.append("NOT EXECUTED", element)
                print(element,"NOT EXECUTED")
    else:
        print(column, " EXCLUDED")
        excluded_line = {"symbol":column}
        excluded_log = excluded_log.append(excluded_line,ignore_index=True)

execution_log.to_csv("/home/sevder/ibtest/current_run/execution_log.csv")
excluded_log.to_csv("/home/sevder/ibtest/current_run/excluded_log.csv")
        


AAPL
SELL
ADSK
BUY
ALNY
BUY
AMED
BUY
ANET
SELL
ATUS
SELL
CE
BUY
CMI
SELL
DLTR
SELL
DOX
BUY
EQIX
BUY
FDX
SELL
FLT
BUY
GH
SELL
IP
SELL
LII
BUY
LK
SELL
MELI
BUY
MKTX
SELL
MLNX  EXCLUDED
NXPI
SELL
OC
SELL
ORLY
BUY
PEN
BUY
RETA
BUY
SWK
SELL
TAP
BUY
TGT
BUY
TYL
BUY
UDR
BUY
VMC
BUY


In [255]:
execution_log

,contract,direction,order,symbol
0,"Stock(symbol='AAPL', exchange='SMART', primary...",SELL,"MarketOrder(action='SELL', account='DU1870227')",AAPL
1,"Stock(symbol='ADSK', exchange='SMART', primary...",BUY,"MarketOrder(action='BUY', account='DU1870227')",ADSK
2,"Stock(symbol='ALNY', exchange='SMART', primary...",BUY,"MarketOrder(action='BUY', account='DU1870227')",ALNY
3,"Stock(symbol='AMED', exchange='SMART', primary...",BUY,"MarketOrder(action='BUY', account='DU1870227')",AMED
4,"Stock(symbol='ANET', exchange='SMART', primary...",SELL,"MarketOrder(action='SELL', account='DU1870227')",ANET
5,"Stock(symbol='ATUS', exchange='SMART', primary...",SELL,"MarketOrder(action='SELL', account='DU1870227')",ATUS
6,"Stock(symbol='CE', exchange='SMART', primaryEx...",BUY,"MarketOrder(action='BUY', account='DU1870227')",CE
7,"Stock(symbol='CMI', exchange='SMART', primaryE...",SELL,"MarketOrder(action='SELL', account='DU1870227')",CMI
8,"Stock(symbol='DLTR', exchange='SMART', primary...",SELL,"MarketOrder(action='SELL', account='DU1870227')",DLTR
9,"Stock(symbol='DOX', exchange='SMART', primaryE...",BUY,"MarketOrder(action='BUY', account='DU1870227')",DOX


In [225]:
ExchangeData

,Stock,primaryExchange
Stock,,
AAPL,AAPL,NASDAQ
MSFT,MSFT,NASDAQ
AMZN,AMZN,NASDAQ
GOOGL,GOOGL,NASDAQ
GOOG,GOOG,NASDAQ
...,...,...
CW,CW,NYSE
PE,PE,NYSE
WF,WF,NYSE


In [209]:
direction

'HOLD'

In [221]:
execution_log

,contract,direction,order,symbol
0,"Stock(symbol='ZTS', exchange='SMART', primaryE...",SELL,"MarketOrder(action='SELL', account='DU1870227')",AAPL
1,"Stock(symbol='ZTS', exchange='SMART', primaryE...",BUY,"MarketOrder(action='BUY', account='DU1870227')",ADSK
2,"Stock(symbol='ZTS', exchange='SMART', primaryE...",BUY,"MarketOrder(action='BUY', account='DU1870227')",ALNY
3,"Stock(symbol='ZTS', exchange='SMART', primaryE...",BUY,"MarketOrder(action='BUY', account='DU1870227')",AMED
4,"Stock(symbol='ZTS', exchange='SMART', primaryE...",SELL,"MarketOrder(action='SELL', account='DU1870227')",ANET
5,"Stock(symbol='ZTS', exchange='SMART', primaryE...",SELL,"MarketOrder(action='SELL', account='DU1870227')",ATUS
6,"Stock(symbol='ZTS', exchange='SMART', primaryE...",BUY,"MarketOrder(action='BUY', account='DU1870227')",CE
7,"Stock(symbol='ZTS', exchange='SMART', primaryE...",SELL,"MarketOrder(action='SELL', account='DU1870227')",CMI
8,"Stock(symbol='ZTS', exchange='SMART', primaryE...",SELL,"MarketOrder(action='SELL', account='DU1870227')",DLTR
9,"Stock(symbol='ZTS', exchange='SMART', primaryE...",BUY,"MarketOrder(action='BUY', account='DU1870227')",DOX


In [208]:
contract

Stock(symbol='ZTS', exchange='SMART', primaryExchange='NYSE', currency='USD')

In [201]:
orders = target_position - CurrentPortfolio

In [202]:
orders

,A,AAL,AAP,AAPL,ABB,ABBV,ABC,ABEV,ABMD,ABT,...,ZBRA,ZEN,ZG,ZION,ZM,ZNGA,ZNH,ZS,ZTO,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2020-06-11,0,0,0,-2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [182]:
#Orders
orders = target_position - CurrentPortfolio

    
        try: 
        amount_portfolio = int(round(float(amount_portfolio)))
        new_amount_portfolio = int(round(float(new_amount_portfolio)))
        print("rounding done")

        # Buy / Sell / Hold
        if new_amount_portfolio>amount_portfolio:
            direction = "BUY"
            trade_amount = new_amount_portfolio - amount_portfolio
        if new_amount_portfolio<amount_portfolio:
            direction = "SELL"
            trade_amount = amount_portfolio - new_amount_portfolio
        if new_amount_portfolio==amount_portfolio:
            direction = "HOLD"
            trade_amount = 0

        if direction!="HOLD":
            #Andy - changed to element as no stock variable set
#            if(element not in excluse):
                try:
                    contract = Stock(element, 'SMART', 'USD', primaryExchange=ExchangeData.loc[element]["primaryExchange"])
                    order = MarketOrder(direction, trade_amount,account=acc_num) #,account=acc_num
                    trade = ib.placeOrder(contract, order)
                    ib.sleep(0.2) # (0.2)
                    print(contract)
                    print(order)
                except:
                    print("NOT EXECUTED",element)
    except:
        print("NOT EXECUTED",element)

A
0
AAL
0
AAP
0
AAPL
0
ABB
0
ABBV
0
ABC
0
ABEV
0
ABMD
0
ABT
0
ACAD
0
ACC
0
ACGL
0
ACGLO
0
ACGLP
0
ACH
0
ACM
0
ACN
0
ADBE
0
ADI
0
ADM
0
ADP
0
ADSK
1
AEE
0
AEG
0
AEM
0
AEP
0
AER
0
AES
0
AFG
0
AFL
0
AGN
0
AGNC
0
AGNCM
0
AGNCN
0
AGR
0
AIG
0
AIV
0
AIZ
0
AIZP
0
AJG
0
AKAM
0
ALB
0
ALC
0
ALGN
0
ALK
0
ALL
0
ALLE
0
ALLY
0
ALNY
14
ALV
0
ALXN
0
AMAT
0
AMCR
0
AMD
0
AME
0
AMED
20
AMGN
0
AMH
0
AMP
0
AMRN
0
AMT
0
AMTD
0
AMX
0
AMZN
0
ANET
-2
ANSS
0
ANTM
0
AON
0
AOS
0
APA
0
APD
0
APH
0
APO
0
APTV
0
AQN
0
ARCC
0
ARE
0
ARES
0
ARGX
0
ARMK
0
ARNC
0
ARW
0
ASML
0
ASND
0
ASX
0
ATH
0
ATHM
0
ATO
0
ATR
0
ATUS
-111
ATVI
0
AU
0
AVB
0
AVGO
0
AVLR
0
AVTR
0
AVY
0
AWK
0
AXP
0
AXTA
0
AYX
0
AZN
0
AZO
0
AZPN
0
BA
0
BABA
0
BAC
0
BAH
0
BAM
0
BAP
0
BATRA
0
BAX
0
BBD
0
BBDO
0
BBL
0
BBVA
0
BBY
0
BCE
0
BCH
0
BCS
0
BDX
0
BEN
0
BEP
0
BFAM
0
BG
0
BGNE
0
BHC
0
BHP
0
BIDU
0
BIIB
0
BILI
0
BIO
0
BIP
0
BK
0
BKI
0
BKNG
0
BKR
0
BLK
0
BLL
0
BMO
0
BMRN
0
BMY
0
BNS
0
BNTX
0
BP
0
BPY
0
BR
0
BRKR
0
BRO
0
BSAC
0
BSBR
0
BSX
0
BTI
0
BUD
0
BURL
0

### INTRADAY

In [304]:
contract = Stock("AAPL", 'SMART', 'USD', primaryExchange=ExchangeData.loc[element]["primaryExchange"])

In [314]:
contract = Stock("AAPL", 'SMART', 'USD', primaryExchange="ISLAND")

In [283]:
ib.reqMktData(contract)

Ticker(contract=Stock(symbol='AAPL', exchange='SMART', primaryExchange='ISLAND', currency='USD'), time=datetime.datetime(2020, 6, 13, 17, 46, 37, 64850, tzinfo=datetime.timezone.utc), marketDataType=3, bid=-1.0, bidSize=0, ask=-1.0, askSize=0, last=338.4, lastSize=1, volume=500377, open=0.0, high=347.8, low=334.22, close=335.9)

In [286]:
ib.reqMktData(contract)

Ticker(contract=Stock(symbol='AAPL', exchange='SMART', primaryExchange='NYSE', currency='USD'), time=datetime.datetime(2020, 6, 13, 17, 46, 55, 955715, tzinfo=datetime.timezone.utc), marketDataType=3, bid=-1.0, bidSize=0, ask=-1.0, askSize=0, last=338.4, lastSize=1, volume=500377, open=0.0, high=347.8, low=334.22, close=335.9)

In [264]:
ib.reqMarketDataType()

TypeError: reqMarketDataType() missing 1 required positional argument: 'marketDataType'

In [291]:
ib.reqMktData(contract)

Ticker(contract=Stock(symbol='AAPL', exchange='SMART', primaryExchange='ISLAND', currency='USD'), time=datetime.datetime(2020, 6, 13, 17, 47, 54, 285358, tzinfo=datetime.timezone.utc), marketDataType=3, bid=-1.0, bidSize=0, ask=-1.0, askSize=0, last=338.4, lastSize=1, volume=500377, open=0.0, high=347.8, low=334.22, close=335.9)

In [315]:
ib.reqMktData(contract)
ib.sleep(0.15)
ib.reqMktData(contract)

Ticker(contract=Stock(symbol='AAPL', exchange='SMART', primaryExchange='ISLAND', currency='USD'), time=datetime.datetime(2020, 6, 13, 17, 55, 51, 11126, tzinfo=datetime.timezone.utc), marketDataType=3, bid=-1.0, bidSize=0, ask=-1.0, askSize=0, last=338.4, lastSize=1, volume=500377, open=0.0, high=347.8, low=334.22, close=335.9)

In [ ]:
NYSE:   334.22 347.8
NASDAQ: 334.22 347.8
ISLAND: 334.22 347.8


In [323]:
request = str("Ticker(contract=Stock(symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD'), time=datetime.datetime(2020, 6, 13, 17, 56, 47, 164807, tzinfo=datetime.timezone.utc), marketDataType=3, bid=-1.0, bidSize=0, ask=-1.0, askSize=0, last=338.4, lastSize=1, volume=500377, open=0.0, high=347.8, low=334.22, close=335.9))")

In [ ]:
line = {"symbol":element, "high":

In [340]:
target_position.columns[1:3]

Index(['MSFT', 'AMZN'], dtype='object')

In [138]:
high = float(request.split("high=")[1].split(",")[0])

IndexError: list index out of range

Peer closed connection


In [37]:
ib.reqMarketDataType(3)

In [137]:
request = str(ib.reqMktData(contract))

In [35]:

contract = Stock("MMM", 'SMART', 'USD', primaryExchange="ISLAND")
ib.reqMktData(contract)

Ticker(contract=Stock(symbol='TSLA', exchange='SMART', primaryExchange='ISLAND', currency='USD'))

In [31]:
ib.reqMktData(contract)

Ticker(contract=Stock(symbol='TSLA', exchange='SMART', primaryExchange='ISLAND', currency='USD'), time=datetime.datetime(2020, 6, 15, 17, 6, 20, 296965, tzinfo=datetime.timezone.utc), marketDataType=3, bid=966.36, bidSize=1, ask=967.01, askSize=1, last=966.36, lastSize=1, volume=82473, open=0.0, high=969.17, low=908.5, close=935.28)

In [44]:
m_data.low

272.77

279.5


In [48]:
contract = Stock('UTX', 'SMART', 'USD')
bars = ib.reqHistoricalData(contract, endDateTime='', durationStr='1 Y',
        barSizeSetting='1 week', whatToShow='MIDPOINT', useRTH=True)
 
# convert to pandas dataframe:
df = util.df(bars)
print(df)

Error 200, reqId 251: No security definition has been found for the request, contract: Stock(symbol='UTX', exchange='SMART', currency='USD')


None


In [86]:
contract = Stock("ADS","SMART","USD")

In [56]:
excluse = ["UTX"]

In [50]:
MyDataLine

,high,last,low,symbol,volume
0,340.30,339.70,332.58,AAPL,176642.0
1,187.97,187.65,184.01,MSFT,145575.0
2,2544.00,2540.24,2508.00,AMZN,19356.0
3,1412.00,1410.93,1387.20,GOOGL,6376.0
4,1411.48,1410.58,1387.92,GOOG,7402.0
...,...,...,...,...,...
56,120.34,120.25,118.29,IBM,25213.0
57,142.38,141.10,139.68,LLY,13508.0
58,340.51,339.98,333.85,TMO,5684.0
59,345.61,344.80,336.63,ASML,2883.0


In [70]:
element not in excluse

False

In [68]:
target_position.columns

Index(['AAPL', 'MSFT', 'AMZN', 'GOOGL', 'GOOG', 'FB', 'BABA', 'V', 'JPM',
       'JNJ',
       ...
       'CUBE', 'CDK', 'AMRN', 'HXL', 'MDU', 'CW', 'PE', 'WF', 'TOL', 'DNKN'],
      dtype='object', length=1000)

In [85]:
ib.cancelMktData(contract)

cancelMktData: No reqId found for contract Stock(symbol='ALLE', exchange='SMART', primaryExchange='NYSE', currency='USD')


In [87]:
m_data = ib.reqMktData(contract)
while m_data.last != m_data.last: ib.sleep(0.01) #Wait until data is in. 
ib.cancelMktData(contract)

In [88]:
m_data

Ticker(contract=Stock(symbol='ADS', exchange='SMART', currency='USD'), time=datetime.datetime(2020, 6, 15, 17, 56, 33, 174423, tzinfo=datetime.timezone.utc), marketDataType=3, bid=50.46, bidSize=3, ask=50.52, askSize=6, last=50.49, lastSize=4, volume=14052, high=51.5, low=48.06, close=52.09)

In [9]:
details = ib.reqContractDetails(Stock("MMM","SMART","USD"))

In [8]:
universe

0       MMM
1       ABT
2      ABBV
3      ABMD
4       ACN
       ... 
500     YUM
501    ZBRA
502     ZBH
503    ZION
504     ZTS
Length: 505, dtype: object

In [6]:
MyDataLine

,high,last,low,symbol,volume
0,158.60,158.54,150.49,MMM,21950.0
1,90.14,90.12,87.05,ABT,28932.0
2,93.70,93.66,90.78,ABBV,38108.0
3,246.48,246.06,236.35,ABMD,2046.0
4,199.98,199.40,194.83,ACN,12576.0
...,...,...,...,...,...
500,91.90,91.74,88.52,YUM,9345.0
501,259.47,258.79,249.10,ZBRA,1756.0
502,127.37,127.37,121.02,ZBH,7040.0
503,36.13,36.13,33.25,ZION,16466.0


Peer closed connection


In [12]:
contract

Stock(symbol='ZTS', exchange='SMART', primaryExchange='NYSE', currency='USD')

In [14]:
ib.reqMarketDataType(3)
m_data = ib.reqMktData(contract)
while m_data.last != m_data.last: ib.sleep(0.01) #Wait until data is in. 
ib.cancelMktData(contract)

In [15]:
m_data

Ticker(contract=Stock(symbol='ZTS', exchange='SMART', primaryExchange='NYSE', currency='USD'), time=datetime.datetime(2020, 6, 16, 7, 53, 49, 16101, tzinfo=datetime.timezone.utc), marketDataType=3, last=0.0, lastSize=0)

Peer closed connection


In [ ]:
##DownloadMarketData
ib.reqMarketDataType(3)
excluse=[]
universe = pickle.load(open('/home/sevder/ibtest/sp500univ.pickle', 'rb'))
SPExchangeData = pd.read_csv("SP500data.csv")
SPExchangeData.index= SPExchangeData["Stock"]
### Simple live data
# stocks = 
MyDataLine=pd.DataFrame()
for element in universe:
    time.sleep(0.05)
    if str(element) not in excluse:
        print(element)
        contract = Stock(element.replace("."," "), 'SMART', 'USD', primaryExchange=SPExchangeData.loc[element]["primaryExchange"])
        try:
            m_data = ib.reqMktData(contract)
            while m_data.last != m_data.last: ib.sleep(0.01) #Wait until data is in. 
            ib.cancelMktData(contract)
            high = m_data.high
            low = m_data.low
            last = m_data.last
            volume = m_data.volume
            line = {"symbol":element,"high":high,"low":low,"last":last,"volume":volume}
        except:
            print("ERROR",element)
        MyDataLine = MyDataLine.append(line,ignore_index=True)
    else:
        print(element,"EXCLUS")
MyDataLine.to_csv("marketdata.csv")
#     ib.sleep(0.5)

In [78]:

### Simple live data

MyDataLine=pd.DataFrame()
for element in target_position.columns:
    if str(element) not in excluse:
        print(element)
        contract = Stock(element, 'SMART', 'USD', primaryExchange=ExchangeData.loc[element]["primaryExchange"])
        try:
            m_data = ib.reqMktData(contract)
            while m_data.last != m_data.last: ib.sleep(0.01) #Wait until data is in. 
            ib.cancelMktData(contract)
            high = m_data.high
            low = m_data.low
            last = m_data.last
            volume = m_data.volume
            line = {"symbol":element,"high":high,"low":low,"last":last,"volume":volume}
        except:
            print("ERROR",element)
        MyDataLine = MyDataLine.append(line,ignore_index=True)
    else:
        print(element,"EXCLUS")
    
#     ib.sleep(0.5)

AAPL
MSFT
AMZN
GOOGL
GOOG
FB
BABA
V
JPM
JNJ
MA
WMT
BAC
PG
TSM
INTC
UNH
T
HD
XOM
KO
DIS
NVS
VZ
TM
IVV
MRK
CVX
CMCSA
PEP
PFE
WFC
CSCO
BA
ADBE
ORCL
NVDA
CHL
C
SAP
NFLX
CRM
MCD
NKE
ABT
MDT
UN
UL
HSBC
BMY
TSLA
PYPL
ABBV
COST
ACN
PM
IBM
LLY
TMO
ASML
NEE
UTX EXCLUS
PTR
AMGN
HON
AZN
AVGO
TOT
UNP
SNY
BUD
TXN
BHP
BBL
LMT
BP
LIN
RY
NVO
AMT
DHR
GE
CHTR
AXP
LFC
GSK
SBUX
QCOM
TD
BTI
LOW
FIS
DEO
CVS
PBR
MMM
RIO
MS
UPS
SNE
BLK
USB
MDLZ
GILD
MO
ENB
GS
ERROR GS
BKNG
CI
FISV
TMUS
SYK
SNP
ADP
INTU
ERROR INTU
EL
ERROR EL
CAT
ERROR CAT
TJX
CME
DUK
ANTM
SPGI
CB
BX
ITUB
TFC
SO
ERROR SO
DCUE
ERROR DCUE
PLD
ERROR PLD
D
SAN
ERROR SAN
CEO
ERROR CEO
BDX
ISRG
CCI
ERROR CCI
ZTS
ERROR ZTS
UBER
ERROR UBER
BNS
ERROR BNS
PNC
ERROR PNC
BAM
ERROR BAM
CNI
ERROR CNI
NOW
ERROR NOW
MU
ERROR MU
CL
ERROR CL
AGN


KeyError: 'AGN'

In [1]:
MyDataLine

NameError: name 'MyDataLine' is not defined

In [16]:
element = "MSFT"
contract = Stock(element, 'SMART', 'USD', primaryExchange=ExchangeData.loc[element]["primaryExchange"])
print(ib.reqMktData(contract))
ib.reqMktData(contract)
request = str(ib.reqTickers(contract))
request = str(ib.reqMktData(contract))

NameError: name 'ExchangeData' is not defined

In [132]:
ib.reqMktData(contract,snapshot=True)

Ticker(contract=Stock(symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD'), time=datetime.datetime(2020, 6, 15, 8, 41, 44, 263526, tzinfo=datetime.timezone.utc), marketDataType=3, bid=330.55, bidSize=1, ask=331.1, askSize=5, last=330.99, lastSize=1, prevBid=330.7, volume=170, open=0.0, close=338.8, ticks=[TickData(time=datetime.datetime(2020, 6, 15, 8, 41, 44, 263526, tzinfo=datetime.timezone.utc), tickType=71, price=330.99, size=1), TickData(time=datetime.datetime(2020, 6, 15, 8, 41, 44, 263526, tzinfo=datetime.timezone.utc), tickType=90, price=0.0, size=0), TickData(time=datetime.datetime(2020, 6, 15, 8, 41, 44, 263526, tzinfo=datetime.timezone.utc), tickType=71, price=330.99, size=1), TickData(time=datetime.datetime(2020, 6, 15, 8, 41, 44, 263526, tzinfo=datetime.timezone.utc), tickType=90, price=0.0, size=0)])

In [127]:
ib.reqTickers(contract)

Error 354, reqId 8: Requested market data is not subscribed.Delayed market data is available.Error&BEST/STK/Top&BEST/STK/Top, contract: Stock(symbol='HD', exchange='SMART', primaryExchange='NYSE', currency='USD')


[Ticker(contract=Stock(symbol='HD', exchange='SMART', primaryExchange='NYSE', currency='USD'))]

In [122]:
request

"Ticker(contract=Stock(symbol='MSFT', exchange='SMART', primaryExchange='NASDAQ', currency='USD'), time=datetime.datetime(2020, 6, 14, 8, 40, 36, 590717, tzinfo=datetime.timezone.utc), marketDataType=3, bid=-1.0, bidSize=0, ask=-1.0, askSize=0, last=188.33, lastSize=10, volume=433740, open=0.0, high=191.72, low=185.18, close=186.27, ticks=[TickData(time=datetime.datetime(2020, 6, 14, 8, 40, 36, 590717, tzinfo=datetime.timezone.utc), tickType=68, price=188.33, size=10), TickData(time=datetime.datetime(2020, 6, 14, 8, 40, 36, 590717, tzinfo=datetime.timezone.utc), tickType=72, price=191.72, size=0), TickData(time=datetime.datetime(2020, 6, 14, 8, 40, 36, 590717, tzinfo=datetime.timezone.utc), tickType=73, price=185.18, size=0), TickData(time=datetime.datetime(2020, 6, 14, 8, 40, 36, 590717, tzinfo=datetime.timezone.utc), tickType=74, price=-1.0, size=433740), TickData(time=datetime.datetime(2020, 6, 14, 8, 40, 36, 590717, tzinfo=datetime.timezone.utc), tickType=75, price=186.27, size=0),

In [129]:
MyDataLine

""


In [350]:
element

'PM'

In [351]:
contract = Stock(element, 'SMART', 'USD', primaryExchange=ExchangeData.loc[element]["primaryExchange"])

In [16]:
request = str(ib.reqMktData(contract))

In [17]:
request

"Ticker(contract=Stock(symbol='FB', exchange='SMART', primaryExchange='NASDAQ', currency='USD'), time=datetime.datetime(2020, 6, 14, 8, 22, 26, 283303, tzinfo=datetime.timezone.utc), marketDataType=3, bid=-1.0, bidSize=0, ask=-1.0, askSize=0, last=228.85, lastSize=1, volume=220916, open=0.0, high=231.66, low=224.5, close=224.43)"

In [356]:
high = float(request.split("high=")[1].split(",")[0])
low = float(request.split("low=")[1].split(",")[0])
last = float(request.split("last=")[1].split(",")[0])

In [357]:
high

72.77

Peer closed connection


In [8]:
#ib.orders()